# Model Builder

Generate the SQLAlchemy ORM model class code for a given table.  
Use this to copy the class code into the models.py file.

In [22]:
from sqlalchemy import inspect

def ORM_Class_Builder(app, db, table_name):
    """
    Generate the SQLAlchemy ORM model class code for a given table.

    Parameters:
        app (Flask): The Flask application.
        db (SQLAlchemy): The SQLAlchemy object.
        table_name (str): The name of the table.

    Returns:
        str: The Python code for the SQLAlchemy ORM model class.
    """
    with app.app_context():
        inspector = inspect(db.engine)

        # Check if the table exists
        if table_name not in inspector.get_table_names():
            raise ValueError(f"Table '{table_name}' does not exist in the database.")

        # Retrieve the table's columns
        columns = inspector.get_columns(table_name)

        # Start building the model class as a string
        class_code = ["from app import db", "", f"class {table_name.capitalize()}(db.Model):"]

        class_code.append(f"\n    __tablename__ = {table_name}\n")


        # Add columns to the model
        for column in columns:
            column_name = column['name']
            column_type = str(column['type'])
            is_primary = column.get('primary_key', False)
            nullable = column.get('nullable', True)

            #print(column_name, column_type)

            # Map SQLAlchemy types
            if "VARCHAR" in column_type.upper() or "CHAR" in column_type.upper():
                column_type = "db.String"
            elif "INTEGER" in column_type.upper():
                column_type = "db.Integer"
            elif "DATE" in column_type.upper() or "date" in column_name.lower():
                column_type = "db.DateTime"  # Map date/datetime column names to datetime
            elif "DATETIME" in column_type.upper():
                column_type = "db.DateTime"
            elif "FLOAT" in column_type.upper():
                column_type = "db.Float"
            elif "REAL" in column_type.upper():
                column_type = "db.Float"
            elif "BOOLEAN" in column_type.upper():
                column_type = "db.Boolean"
            elif "NUMERIC" in column_type.upper():
                column_type = "db.Numeric"
            else:
                column_type = "db.Text"

            #print(f"\t{column_type}")
            # Build the column definition
            column_line = f"    {column_name} = db.Column({column_type}"
            if is_primary:
                column_line += ", primary_key=True"
            if not nullable:
                column_line += ", nullable=False"
            column_line += ")"
            class_code.append(column_line)

        # Add the __repr__ method
        class_code.append("")
        class_code.append("    def __repr__(self):")
        class_code.append(f"        return f'<{table_name.capitalize()} {{self.id}}>'")

        # Join the class code into a single string
        return "\n".join(class_code)

# Example usage
# with app.app_context():
#     print(generate_model_code(app, db, 'your_table_name'))


In [23]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = 'key_for_csrf_token_required_for_form_data'

db_path = os.path.abspath("synthetic_data/teleCC.sqlite")
app.config['SQLALCHEMY_DATABASE_URI'] = f"sqlite:///{db_path}"

# Track modifications is often unnecessary and can be disabled
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

In [24]:
from sqlalchemy import inspect

with app.app_context():
    inspector = inspect(db.engine)

    for table in inspector.get_table_names():
        print(table)

patient_teleCC
patient_teleCC_history
patient_teleCC_state
station_lookup


In [25]:
table_name = 'station_lookup'
class_code = ORM_Class_Builder(app, db, table_name)
print(class_code)

from app import db

class Station_lookup(db.Model):

    __tablename__ = station_lookup

    station_number = db.Column(db.Integer, primary_key=True)
    station_name = db.Column(db.Text)

    def __repr__(self):
        return f'<Station_lookup {self.id}>'
